# hsh폴더에 대해서만 train, test 
 일반화를 하지 않고, 개인화에 초점을 맞춤 -> 한 서브젝트에 대해서만 분류 진행

 이후 대조해 볼 것으로 모든 서브젝트에 대해서 분류 진행 -> 전의 경우는 서브젝트에 대해서 loso로 서브젝트 분리 후 train, test를 진행했었음

In [3]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# ResNet18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(5, 7), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 3), padding='same')(x)
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# ResNet 블록 생성
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# MLP 모델 생성
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation=mish)(input_layer)
    x = layers.Dense(64, activation=mish)(x)
    x = layers.Dense(3, activation=mish)(x)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

valid_labels = [1, 5, 9, 19, 23, 27, 37, 41, 45]

def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

from sklearn.model_selection import train_test_split

# hsh 폴더에서 train/test 분리 및 학습 진행
def train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject, test_size=0.2):
    print(f"Training and testing from folder: {subject}")
    
    # hsh 폴더 데이터 로드
    folder = os.path.join(folder_path, subject)
    csv_file = os.path.join(csv_path, f"{subject}.csv")
    image_paths = glob.glob(os.path.join(folder, '*.jpg'))
    csv_data = pd.read_csv(csv_file) if os.path.exists(csv_file) else pd.DataFrame(columns=['Session', 'Point'])
    csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 이미지와 라벨 처리
    data = []
    for img in image_paths:
        session, point = extract_session_and_point(os.path.basename(img))
        if point in valid_labels:
            unique_filename = f"{subject}_{os.path.basename(img)}"
            data.append({
                'Filename': os.path.abspath(img),
                'UniqueFilename': unique_filename,
                'Session': session,
                'Point': point
            })
    df = pd.DataFrame(data)
    merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
    merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
    merged = merged[merged['Point'].isin(valid_labels)]
    
    # Train/Test Split
    train_df, test_df = train_test_split(merged, test_size=test_size, random_state=42, stratify=merged['Point'])
    
    train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
    train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
    
    test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
    test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
    
    # 모델 생성
    right_eye_model = create_resnet18_model((128, 128, 3))
    left_eye_model = create_resnet18_model((128, 128, 3))
    mlp_model = create_mlp_model(train_features.shape[1:])
    combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])
    x = layers.Dense(256, activation=mish)(combined_input)
    x = layers.Dropout(0.5)(x)

    output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
    combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = combined_model.fit(
        [train_images_array, train_images_array, train_features], train_labels,
        validation_data=([test_images_array, test_images_array, test_features], test_labels),
        epochs=50,
        batch_size=1,
        verbose=1
    )
    
    # 평가
    predictions = combined_model.predict([test_images_array, test_images_array, test_features])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(test_labels, axis=1)

    loss, accuracy = combined_model.evaluate([test_images_array, test_images_array, test_features], test_labels)
    print(f"Testing accuracy for folder {subject}: {accuracy:.4f}")
    
    # 결과 저장
    results = []
    for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
        test_df['Filename'], test_features, predicted_classes, actual_classes)):
        results.append({
            'Subject': subject,
            'Test Accuracy': accuracy,
            'Test Loss': loss,
            'Image File': image_path,
            'Feature': feature.tolist(),
            'Predicted Class': pred_class,
            'Actual Class': actual_class
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(csv_path, f"0102_results_{subject}.csv"), index=False, encoding='utf-8')
    print(f"Results for folder {subject} saved.")
    
    # 모델 저장
    model_save_path = os.path.join(csv_path, f"0102_model_{subject}.h5")
    combined_model.save(model_save_path)
    print(f"Model saved at: {model_save_path}")

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
subject = 'hsh'

# 동일 폴더에서 train/test 분리 및 실행
train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject)

Training and testing from folder: hsh
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_139', 'keras_tensor_204', 'keras_tensor_269']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


360/360 ━━━━━━━━━━━━━━━━━━━━ 94s 214ms/step - accuracy: 0.1441 - loss: 4.2612 - val_accuracy: 0.1556 - val_loss: 2.1907
Epoch 2/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 76s 211ms/step - accuracy: 0.4059 - loss: 1.4616 - val_accuracy: 0.4000 - val_loss: 1.4715
Epoch 3/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 76s 211ms/step - accuracy: 0.5766 - loss: 0.8945 - val_accuracy: 0.6444 - val_loss: 1.6981
Epoch 4/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 76s 211ms/step - accuracy: 0.7649 - loss: 0.6382 - val_accuracy: 0.8889 - val_loss: 0.3299
Epoch 5/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 76s 212ms/step - accuracy: 0.9324 - loss: 0.2057 - val_accuracy: 0.7556 - val_loss: 0.5023
Epoch 6/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 76s 212ms/step - accuracy: 0.9449 - loss: 0.1962 - val_accuracy: 0.7333 - val_loss: 1.2229
Epoch 7/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 76s 212ms/step - accuracy: 0.8839 - loss: 0.2702 - val_accuracy: 0.9778 - val_loss: 0.0945
Epoch 8/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 80s 222ms/step - accuracy: 0.9954 - loss: 0.0133 - val

KeyboardInterrupt: 

In [4]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# ResNet18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(5, 7), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(2, 3), padding='same')(x)
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# ResNet 블록 생성
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# MLP 모델 생성
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation=mish)(input_layer)
    x = layers.Dense(64, activation=mish)(x)
    x = layers.Dense(3, activation=mish)(x)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

valid_labels = [1, 5, 9, 12, 16, 19, 23, 27, 30, 34, 37, 41, 45]

def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

from sklearn.model_selection import train_test_split

# hsh 폴더에서 train/test 분리 및 학습 진행
def train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject, test_size=0.2):
    print(f"Training and testing from folder: {subject}")
    
    # hsh 폴더 데이터 로드
    folder = os.path.join(folder_path, subject)
    csv_file = os.path.join(csv_path, f"{subject}.csv")
    image_paths = glob.glob(os.path.join(folder, '*.jpg'))
    csv_data = pd.read_csv(csv_file) if os.path.exists(csv_file) else pd.DataFrame(columns=['Session', 'Point'])
    csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 이미지와 라벨 처리
    data = []
    for img in image_paths:
        session, point = extract_session_and_point(os.path.basename(img))
        if point in valid_labels:
            unique_filename = f"{subject}_{os.path.basename(img)}"
            data.append({
                'Filename': os.path.abspath(img),
                'UniqueFilename': unique_filename,
                'Session': session,
                'Point': point
            })
    df = pd.DataFrame(data)
    merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
    merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
    merged = merged[merged['Point'].isin(valid_labels)]
    
    # Train/Test Split
    train_df, test_df = train_test_split(merged, test_size=test_size, random_state=42, stratify=merged['Point'])
    
    train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
    train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
    
    test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
    test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
    
    # 모델 생성
    right_eye_model = create_resnet18_model((128, 128, 3))
    left_eye_model = create_resnet18_model((128, 128, 3))
    mlp_model = create_mlp_model(train_features.shape[1:])
    combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])
    x = layers.Dense(256, activation=mish)(combined_input)
    x = layers.Dropout(0.5)(x)

    output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
    combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = combined_model.fit(
        [train_images_array, train_images_array, train_features], train_labels,
        validation_data=([test_images_array, test_images_array, test_features], test_labels),
        epochs=50,
        batch_size=1,
        verbose=1
    )
    
    # 평가
    predictions = combined_model.predict([test_images_array, test_images_array, test_features])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(test_labels, axis=1)

    loss, accuracy = combined_model.evaluate([test_images_array, test_images_array, test_features], test_labels)
    print(f"Testing accuracy for folder {subject}: {accuracy:.4f}")
    
    # 결과 저장
    results = []
    for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
        test_df['Filename'], test_features, predicted_classes, actual_classes)):
        results.append({
            'Subject': subject,
            'Test Accuracy': accuracy,
            'Test Loss': loss,
            'Image File': image_path,
            'Feature': feature.tolist(),
            'Predicted Class': pred_class,
            'Actual Class': actual_class
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(csv_path, f"0102_results_{subject}.csv"), index=False, encoding='utf-8')
    print(f"Results for folder {subject} saved.")
    
    # 모델 저장
    model_save_path = os.path.join(csv_path, f"0102_model_{subject}.h5")
    combined_model.save(model_save_path)
    print(f"Model saved at: {model_save_path}")

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
subject = 'hsh'

# 동일 폴더에서 train/test 분리 및 실행
train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject)

Training and testing from folder: hsh
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_278', 'keras_tensor_343', 'keras_tensor_408']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


520/520 ━━━━━━━━━━━━━━━━━━━━ 128s 216ms/step - accuracy: 0.0699 - loss: 3.7767 - val_accuracy: 0.2615 - val_loss: 4.2111
Epoch 2/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 110s 211ms/step - accuracy: 0.3106 - loss: 1.8737 - val_accuracy: 0.4308 - val_loss: 2.3839
Epoch 3/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 109s 210ms/step - accuracy: 0.6179 - loss: 0.9717 - val_accuracy: 0.6308 - val_loss: 1.2357
Epoch 4/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 109s 210ms/step - accuracy: 0.7932 - loss: 0.6353 - val_accuracy: 0.6308 - val_loss: 1.5662
Epoch 5/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 109s 209ms/step - accuracy: 0.9059 - loss: 0.2960 - val_accuracy: 0.7769 - val_loss: 0.7745
Epoch 6/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 109s 210ms/step - accuracy: 0.9233 - loss: 0.2633 - val_accuracy: 0.7923 - val_loss: 0.6331
Epoch 7/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 109s 210ms/step - accuracy: 0.9480 - loss: 0.1990 - val_accuracy: 0.7769 - val_loss: 0.8418
Epoch 8/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 109s 210ms/step - accuracy: 0.9232 - loss: 0.23

Testing accuracy for folder hsh: 0.8231
Results for folder hsh saved.
Model saved at: C:\Users\admin\Desktop\sihoon\webcam\results\0102_model_hsh.h5


## Resnet 18, 9class, 13class

In [6]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# ResNet18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# ResNet 블록 생성
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# MLP 모델 생성
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation=mish)(input_layer)
    x = layers.Dense(64, activation=mish)(x)
    x = layers.Dense(3, activation=mish)(x)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

valid_labels = [1, 5, 9, 19, 23, 27, 37, 41, 45]

def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

from sklearn.model_selection import train_test_split

# hsh 폴더에서 train/test 분리 및 학습 진행
def train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject, test_size=0.2):
    print(f"Training and testing from folder: {subject}")
    
    # hsh 폴더 데이터 로드
    folder = os.path.join(folder_path, subject)
    csv_file = os.path.join(csv_path, f"{subject}.csv")
    image_paths = glob.glob(os.path.join(folder, '*.jpg'))
    csv_data = pd.read_csv(csv_file) if os.path.exists(csv_file) else pd.DataFrame(columns=['Session', 'Point'])
    csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 이미지와 라벨 처리
    data = []
    for img in image_paths:
        session, point = extract_session_and_point(os.path.basename(img))
        if point in valid_labels:
            unique_filename = f"{subject}_{os.path.basename(img)}"
            data.append({
                'Filename': os.path.abspath(img),
                'UniqueFilename': unique_filename,
                'Session': session,
                'Point': point
            })
    df = pd.DataFrame(data)
    merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
    merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
    merged = merged[merged['Point'].isin(valid_labels)]
    
    # Train/Test Split
    train_df, test_df = train_test_split(merged, test_size=test_size, random_state=42, stratify=merged['Point'])
    
    train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
    train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
    
    test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
    test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
    
    # 모델 생성
    right_eye_model = create_resnet18_model((128, 128, 3))
    left_eye_model = create_resnet18_model((128, 128, 3))
    mlp_model = create_mlp_model(train_features.shape[1:])
    combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])
    x = layers.Dense(256, activation=mish)(combined_input)
    x = layers.Dropout(0.5)(x)

    output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
    combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = combined_model.fit(
        [train_images_array, train_images_array, train_features], train_labels,
        validation_data=([test_images_array, test_images_array, test_features], test_labels),
        epochs=50,
        batch_size=1,
        verbose=1
    )
    
    # 평가
    predictions = combined_model.predict([test_images_array, test_images_array, test_features])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(test_labels, axis=1)

    loss, accuracy = combined_model.evaluate([test_images_array, test_images_array, test_features], test_labels)
    print(f"Testing accuracy for folder {subject}: {accuracy:.4f}")
    
    # 결과 저장
    results = []
    for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
        test_df['Filename'], test_features, predicted_classes, actual_classes)):
        results.append({
            'Subject': subject,
            'Test Accuracy': accuracy,
            'Test Loss': loss,
            'Image File': image_path,
            'Feature': feature.tolist(),
            'Predicted Class': pred_class,
            'Actual Class': actual_class
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(csv_path, f"0102_results_{subject}.csv"), index=False, encoding='utf-8')
    print(f"Results for folder {subject} saved.")
    
    # 모델 저장
    model_save_path = os.path.join(csv_path, f"0102_model_{subject}.h5")
    combined_model.save(model_save_path)
    print(f"Model saved at: {model_save_path}")

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
subject = 'hsh'

# 동일 폴더에서 train/test 분리 및 실행
train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject)

Training and testing from folder: hsh
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_556', 'keras_tensor_621', 'keras_tensor_686']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


360/360 ━━━━━━━━━━━━━━━━━━━━ 78s 168ms/step - accuracy: 0.2126 - loss: 3.3935 - val_accuracy: 0.1889 - val_loss: 1.9803
Epoch 2/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 59s 163ms/step - accuracy: 0.6152 - loss: 1.0700 - val_accuracy: 0.1889 - val_loss: 3.6414
Epoch 3/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 59s 164ms/step - accuracy: 0.6717 - loss: 0.7357 - val_accuracy: 0.7333 - val_loss: 0.6641
Epoch 4/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 59s 165ms/step - accuracy: 0.7566 - loss: 0.6178 - val_accuracy: 0.8556 - val_loss: 0.4814
Epoch 5/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 61s 169ms/step - accuracy: 0.8310 - loss: 0.3939 - val_accuracy: 0.8667 - val_loss: 0.6367
Epoch 6/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 60s 166ms/step - accuracy: 0.8750 - loss: 0.3835 - val_accuracy: 0.8333 - val_loss: 0.6050
Epoch 7/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 59s 165ms/step - accuracy: 0.9127 - loss: 0.3093 - val_accuracy: 0.8333 - val_loss: 0.5625
Epoch 8/50
360/360 ━━━━━━━━━━━━━━━━━━━━ 61s 169ms/step - accuracy: 0.9694 - loss: 0.0846 - val

Testing accuracy for folder hsh: 0.6111
Results for folder hsh saved.
Model saved at: C:\Users\admin\Desktop\sihoon\webcam\results\0102_model_hsh.h5


In [7]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# ResNet18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# ResNet 블록 생성
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# MLP 모델 생성
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation=mish)(input_layer)
    x = layers.Dense(64, activation=mish)(x)
    x = layers.Dense(3, activation=mish)(x)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

valid_labels = [1, 5, 9, 12, 16, 19, 23, 27, 30, 34, 37, 41, 45]

def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

from sklearn.model_selection import train_test_split

# hsh 폴더에서 train/test 분리 및 학습 진행
def train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject, test_size=0.2):
    print(f"Training and testing from folder: {subject}")
    
    # hsh 폴더 데이터 로드
    folder = os.path.join(folder_path, subject)
    csv_file = os.path.join(csv_path, f"{subject}.csv")
    image_paths = glob.glob(os.path.join(folder, '*.jpg'))
    csv_data = pd.read_csv(csv_file) if os.path.exists(csv_file) else pd.DataFrame(columns=['Session', 'Point'])
    csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 이미지와 라벨 처리
    data = []
    for img in image_paths:
        session, point = extract_session_and_point(os.path.basename(img))
        if point in valid_labels:
            unique_filename = f"{subject}_{os.path.basename(img)}"
            data.append({
                'Filename': os.path.abspath(img),
                'UniqueFilename': unique_filename,
                'Session': session,
                'Point': point
            })
    df = pd.DataFrame(data)
    merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
    merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
    merged = merged[merged['Point'].isin(valid_labels)]
    
    # Train/Test Split
    train_df, test_df = train_test_split(merged, test_size=test_size, random_state=42, stratify=merged['Point'])
    
    train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
    train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
    
    test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
    test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
    
    # 모델 생성
    right_eye_model = create_resnet18_model((128, 128, 3))
    left_eye_model = create_resnet18_model((128, 128, 3))
    mlp_model = create_mlp_model(train_features.shape[1:])
    combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])
    x = layers.Dense(256, activation=mish)(combined_input)
    x = layers.Dropout(0.5)(x)

    output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
    combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = combined_model.fit(
        [train_images_array, train_images_array, train_features], train_labels,
        validation_data=([test_images_array, test_images_array, test_features], test_labels),
        epochs=50,
        batch_size=1,
        verbose=1
    )
    
    # 평가
    predictions = combined_model.predict([test_images_array, test_images_array, test_features])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(test_labels, axis=1)

    loss, accuracy = combined_model.evaluate([test_images_array, test_images_array, test_features], test_labels)
    print(f"Testing accuracy for folder {subject}: {accuracy:.4f}")
    
    # 결과 저장
    results = []
    for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
        test_df['Filename'], test_features, predicted_classes, actual_classes)):
        results.append({
            'Subject': subject,
            'Test Accuracy': accuracy,
            'Test Loss': loss,
            'Image File': image_path,
            'Feature': feature.tolist(),
            'Predicted Class': pred_class,
            'Actual Class': actual_class
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(csv_path, f"0102_results_{subject}.csv"), index=False, encoding='utf-8')
    print(f"Results for folder {subject} saved.")
    
    # 모델 저장
    model_save_path = os.path.join(csv_path, f"0102_model_{subject}.h5")
    combined_model.save(model_save_path)
    print(f"Model saved at: {model_save_path}")

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
subject = 'hsh'

# 동일 폴더에서 train/test 분리 및 실행
train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject)

Training and testing from folder: hsh
Epoch 1/50


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_695', 'keras_tensor_760', 'keras_tensor_825']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


520/520 ━━━━━━━━━━━━━━━━━━━━ 253s 275ms/step - accuracy: 0.1565 - loss: 3.8154 - val_accuracy: 0.4231 - val_loss: 1.7898
Epoch 2/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 158s 303ms/step - accuracy: 0.5072 - loss: 1.3620 - val_accuracy: 0.7231 - val_loss: 0.8345
Epoch 3/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 201s 301ms/step - accuracy: 0.6054 - loss: 1.0471 - val_accuracy: 0.5846 - val_loss: 1.0482
Epoch 4/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 157s 302ms/step - accuracy: 0.7595 - loss: 0.6796 - val_accuracy: 0.8462 - val_loss: 0.5744
Epoch 5/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 161s 309ms/step - accuracy: 0.8845 - loss: 0.3233 - val_accuracy: 0.9077 - val_loss: 0.3480
Epoch 6/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 157s 302ms/step - accuracy: 0.8648 - loss: 0.3724 - val_accuracy: 0.8308 - val_loss: 0.5852
Epoch 7/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 202s 300ms/step - accuracy: 0.9185 - loss: 0.3970 - val_accuracy: 0.9462 - val_loss: 0.2462
Epoch 8/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 156s 300ms/step - accuracy: 0.9243 - loss: 0.22

4/5 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/stepWARNING:tensorflow:6 out of the last 13 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002362FA2D580> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


5/5 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step 
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 358ms/step - accuracy: 0.9132 - loss: 0.7261


Testing accuracy for folder hsh: 0.9154
Results for folder hsh saved.
Model saved at: C:\Users\admin\Desktop\sihoon\webcam\results\0102_model_hsh.h5
